In [258]:
import numpy as np
import matplotlib.pyplot as plt
# This part calls for the tex package with Palantino font family 
# Note latex should be installed in the system to run this
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Palatino"
})
from math import *

In [259]:
meshlim = 100
meshsize = 3001
x = np.linspace(-meshlim, meshlim, meshsize)
y = np.linspace(-meshlim, meshlim, meshsize)
# full coordinate arrays
xx, yy = np.meshgrid(x, y)
zz = np.exp(-(xx**2 + yy**2))
xx.shape, yy.shape, zz.shape

((3001, 3001), (3001, 3001), (3001, 3001))

In [260]:
Delta = 2*meshlim/(meshsize-1)
kx = (2*np.pi/(meshsize*Delta))*np.arange(-(meshsize-1)/2,(meshsize-1)/2+1)
ky = (2*np.pi/(meshsize*Delta))*np.arange(-(meshsize-1)/2,(meshsize-1)/2+1)
kxx, kyy = np.meshgrid(kx, ky)
kzz = 0.5*np.exp(-(kxx**2+kyy**2)/4)

In [261]:
omegazz = np.fft.fft2(zz,norm="ortho")
omegazz = np.fft.fftshift(omegazz)

In [262]:
factor = (Delta**2)*(meshsize/(2*np.pi))*np.exp(-1j*(kxx*np.min(x)+kyy*np.min(y)))
ft_omegazz = factor*omegazz

In [ ]:
fig1 = plt.figure(figsize=(10,10))
ax1 = plt.axes(projection='3d')
ax1.contour3D(kxx, kyy, ft_omegazz,100)
ax1.view_init(15, 35)
plt.show()

In [ ]:
fig2 = plt.figure(figsize=(10,10))
ax2 = plt.axes(projection='3d')
ax2.contour3D(kx, ky, kzz, 100)
ax2.view_init(15, 35)
plt.show()

In [ ]:
np.allclose(ft_omegazz,kzz)